In [1]:
from IPython.display import clear_output
import time
import copy

def hasUnmarkedEdge(matrix, size):
    for i in range(size):
        if np.sum(matrix[i]) > 0:
            return True
    return False

In [2]:
class Graph:
    def __init__(self, verticesTotal):
        if verticesTotal <= 0:
            verticesTotal = 1

        self.verticesTotal = verticesTotal
        self.table = [([0] * verticesTotal) for i in range(verticesTotal)]
    
    def print(self):
        for i in range(self.verticesTotal):
            print(self.table[i])

    def addVertex(self):
        for i in range(self.verticesTotal):
            self.table[i].append(0)
            
        self.verticesTotal += 1
        self.table.append([0] * self.verticesTotal)

    def deleteVertex(self, vertex=-1):
        if self.checkVertexNumber(vertex) == False:
            print("deleteVertex: Vertex is out of range")
            return

        index = vertex - 1
        
        for i in range(self.verticesTotal):
            self.table[i].pop(index);

        self.verticesTotal -= 1
        self.table.pop(index);

    def addEdge(self, vertex1=-1, vertex2=-1, weight=1):
        if not self.checkVertexNumber(vertex1) or not self.checkVertexNumber(vertex2):
            print("addEdge: Any of vertices is out of range!")
            return

        if vertex1 == vertex2:
            print("I can not do this!")
            return

        index1 = vertex1 - 1
        index2 = vertex2 - 1

        self.table[index1][index2] = weight
        self.table[index2][index1] = weight

    def deleteEdge(self, vertex1=-1, vertex2=-1):
        if self.checkVertexNumber(vertex1) == False or self.checkVertexNumber(vertex2) == False:
            print("deleteEdge: Any of vertices is out of range!")
            return

        if vertex1 == vertex2:
            print("I can not do this!")
            return

        index1 = vertex1 - 1
        index2 = vertex2 - 1

        if self.table[index1][index2] == 0 and self.table[index2][index1] == 0:
            print("There is no such edge in graph!")
            return

        self.table[index1][index2] = 0
        self.table[index2][index1] = 0

    def getVertexDegree(self, vertex):
        if self.checkVertexNumber(vertex) == False:
            print("getVertexDegree: Vertex is out of range")
            return

        return np.sum(self.table[vertex - 1])

    def getIndexOfFirstVertexWithOddDegree(self):
        for i in range(self.verticesTotal):
            if self.getVertexDegree(i + 1) % 2 != 0:
                return i
        return 0

    def checkVertexNumber(self, number):
        if number > 0 and number <= self.verticesTotal:
            return True
        return False
    
    def getEdgesWithWeight(self):
        tableCopy = copy.deepcopy(self.table)
        edges = []

        for i in range(self.verticesTotal):
            for k in range(i, self.verticesTotal):
                if self.table[i][k] != 0:
                    edges.append([i, k, self.table[i][k]])
                    self.deleteEdge(i+1, k+1)
            
        self.table = tableCopy
        return edges
    
    def isCyclicUtil(self,v,visited,parent): 
        visited[v]= True
        
        neighbours = [j for j in range(self.verticesTotal) if self.table[v][j] != 0]

        for i in neighbours: 
            if  visited[i]==False :  
                if self.isCyclicUtil(i,visited,v): 
                    return True

            elif  parent != i: 
                return True
          
        return False
           
   
    def isCyclic(self): 
        visited =[False]* self.verticesTotal 

        for i in range(self.verticesTotal): 
            if visited[i] == False: 
                if(self.isCyclicUtil(i,visited,-1))== True: 
                    return True
          
        return False
        
    def prim(self, startWith=1):
        tableCopy = copy.deepcopy(self.table)
        
        U = [startWith-1]
        vertices = [i for i in range(self.verticesTotal)]
        
        # while vertices not in U exist
        while [j for j in vertices if j not in U]:
            countUIndex = -1
            vertex = U[countUIndex]
            
            while not [i for i in self.table[vertex] if i != 0]:
                U.append(U[countUIndex - 1])
                countUIndex -= 2
                vertex = U[countUIndex]
            
            m = self.table[vertex].index(min(i for i in self.table[vertex] if i != 0))
            
            self.deleteEdge(vertex + 1, m + 1)
            
            if m not in U:
                U.append(m)
    
        self.table = tableCopy
        
        return [i+1 for i in U]

    def kruskal(self):
        sortedEdges = sorted(self.getEdgesWithWeight(), key=lambda item: item[2])
        newGraph = Graph(self.verticesTotal)
        
        for edge in sortedEdges:
            newGraph.addEdge(edge[0]+1, edge[1]+1, edge[2])
            
            if newGraph.isCyclic():
                newGraph.deleteEdge(edge[0]+1, edge[1]+1)
        
        return [[i[0]+1, i[1]+1, i[2]] for i in newGraph.getEdgesWithWeight()]

In [3]:
# LOGIC
isEnd = False
g = Graph(1)
prim = []
kruskal = []
concatedGraph = None
haveConcated = False

while not isEnd:
    clear_output()
    
    print("Graph:")
    g.print()
    print("\n")
    
    if prim:
        print("Prim's path:\n", prim, "\n")
        
    if kruskal:
        print("Kruskal's vertices", kruskal, "\n")

    if haveConcated:
        concatedGraph.print()
        
    print("1. Add a vertex\n" +
          "2. Delete a vertex\n" +
          "3. Add an edge\n" +          
          "4. Delete an edge\n\n" +
          "5. Get Prim's path\n" +
          "6. Get Kraskal's path\n" +
          "7. Concate Prim's and Kruskal's graphs\n\n" +
          "9. Presetted graph\n" +
          "q. To quit the program\n")

    inputValue = input("Enter a number or 'q': ")
    
    if inputValue == '1':
        g.addVertex()
        
    elif inputValue == '2':
        g.deleteVertex(int(input("Enter a vertex to delete: ")))
        
    elif inputValue == '3':
        g.addEdge(int(input("Enter number of first vertex: ")), 
                  int(input("Enter number of second vertex: ")), 
                  int(input("Enter weight: ")))
        
    elif inputValue == '4':
        g.deleteEdge(int(input("Enter number of first vertex: ")), 
                     int(input("Enter number of second vertex: ")))
        
    elif inputValue == '5':
        prim = g.prim(int(input("Start with: ")))
    
    elif inputValue == '6':
        kruskal = g.kruskal()
        
    elif inputValue == '7':
        if kruskal and prim:
            concatedGraph = Graph(g.verticesTotal)
            
            for i in range(len(prim) - 1):
                concatedGraph.addEdge(prim[i], prim[i+1], g.table[i][i+1])
            
            for i in kruskal:
                concatedGraph.addEdge(i[0], i[1], i[2])
            
            haveConcated = True
        else:
            print("¯\_(ツ)_/¯")
            time.sleep(1)

    elif inputValue == '9':
        g = Graph(7)
        g.addEdge(1,2,1)
        g.addEdge(1,7,1)
        g.addEdge(2,5,2)
        g.addEdge(2,7,1)
        g.addEdge(3,4,3)
        g.addEdge(3,7,5)
        g.addEdge(4,6,4)
        g.addEdge(5,7,2)
        g.addEdge(5,6,2)
        g.addEdge(6,7,2)
        
    elif inputValue == 'q':
        isEnd = True
        
    else:
        print("¯\_(ツ)_/¯")
        time.sleep(1)


Graph:
[0]


1. Add a vertex
2. Delete a vertex
3. Add an edge
4. Delete an edge

5. Get Prim's path
6. Get Kraskal's path
7. Concate Prim's and Kruskal's graphs

9. Presetted graph
q. To quit the program

Enter a number or 'q': q
